In [5]:
!pip install -U adapter-transformers
!pip install datasets

  Using cached adapter_transformers-2.2.0-py3-none-any.whl (2.9 MB)
     |████████████████████████████████| 748 kB 5.3 MB/s eta 0:00:01
  Using cached sacremoses-0.0.46-py3-none-any.whl (895 kB)
     |████████████████████████████████| 14.8 MB 62.3 MB/s eta 0:00:01
  Using cached huggingface_hub-0.2.1-py3-none-any.whl (61 kB)
     |████████████████████████████████| 3.3 MB 56.2 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4
  Using cached datasets-1.16.1-py3-none-any.whl (298 kB)
     |████████████████████████████████| 1.1 MB 4.0 MB/s eta 0:00:01
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
     |████████████████████████████████| 86 kB 661 kB/s  eta 0:00:01
     |████████████████████████████████| 106 kB 71.7 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 73.8 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 71.9 MB/s eta 0:00:

In [6]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("imdb")
dataset.num_rows

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [4]:
train_dataset, valid_dataset= dataset["train"].train_test_split(test_size=0.2).values()
dataset = DatasetDict({"train":train_dataset,"test":dataset["test"], "validation":valid_dataset})
dataset.num_rows

Loading cached split indices for dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-6c12e3fce669ca46.arrow and /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-b46059cb46649fd5.arrow


{'train': 40000, 'test': 5000, 'validation': 5000}

In [5]:
dataset["train"][0]

{'text': 'This movie is told through the eyes of a young teacher at a catholic school, watching as the RAWANDAN genocide un-furls around him.<br /><br />The movie starts off with a brief explanation about the past history and rivalry of Rawanda. Then it jumps to the story as told through the eyes of a young idealistic "NEW-COMER" a young teacher who doesn\'t take life or the situation too seriously. As he and the driver approach a road-block he plays around with his drivers I.D. not realizing that this is a serious moment and that if the driver can\'t identify himself as being of the right tribe to the soldiers they\'ll be killed. And thats how he treats the unfolding story of chaos and unfolding around him. Suddenly realizes that every Rawandan (including his driver) is involved and that the Europeans soldiers and tourists cannot and will not help. The media cameras cannot stop machete\'s, and there\'s too many machete wielding militia-men too shoot. the title comes from the armies ca

In [4]:
max_l = max([len(d['text']) for d in dataset["train"]])

In [ ]:
!pip3 install torch torchvision

In [6]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=512, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset.rename_column_("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

  0%|          | 0/40 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: rename_column_ is deprecated and will be removed in the next major version of datasets. Use DatasetDict.rename_column instead.
  if sys.path[0] == '':


In [7]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=2,
)
model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

In [8]:
# Add a new adapter
model.add_adapter("imdb")
# Add a matching classification head
model.add_classification_head(
    "imdb",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"}
  )
# Activate the adapter
model.train_adapter("imdb")

In [9]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 40000
  Num Epochs = 6
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7500


In [1]:
trainer.evaluate()

NameError: name 'trainer' is not defined

In [ ]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

classifier("This is awesome!")

In [ ]:
model.save_adapter("./final_adapter", "rotten_tomatoes")

!ls -lh final_adapter